# Preprocessing

In [1]:
import io
import os
import sys
import PyPDF2
from pathlib2 import Path
from shutil import copyfile
import textract
import parallel_run
from words import splitted_words, replace_digits

Sposta i files che soddisfano un dato criterio da un albero di sottocartelle ad una cartella

In [5]:
def move_flattened_files(src_dir, out_dir, filt):
    p = Path(src_dir)
    if p.is_dir():
        for f in p.iterdir():
            if f.is_file() and filt(f):
                copyfile(str(f.absolute()),str(out_dir + u'/' + f.name))
            else:
                move_flattened_files(str(f.absolute()), out_dir, filt)

def file_filter(f):
    return f.name.endswith(u'001.pdf') and os.path.getsize(str(f.absolute()))<(1024**2)

In [ ]:
move_flattened_files('../raw_c','../atti_costitutivi2', lambda f: f.name.endswith('001.pdf') and os.path.getsize(f.name)<(1024**2))

In [6]:
move_flattened_files(u'../raw_a11',u'../atti_non_costitutivi2', file_filter)

Toglie i files con un certo criterio (dimensione maggiore al MB)

In [58]:
def delete_files(dir_name, pred):
    p = Path(dir_name)
    for f in p.iterdir():
        if (not f.is_dir()) and pred(str(f.absolute())):
            print str(f.absolute())
            os.remove(str(f.absolute()))

In [ ]:
delete_files('../atti_costitutivi', lambda f: os.path.getsize(f)>(1024**2))

Utils

In [16]:
orig = [f[:-4] for f in os.listdir("../atti_costitutivi")]
orig2 = [f[:-4] for f in os.listdir("../atti_non_costitutivi")]
f1 = [f[:-4] for f in os.listdir("../atti_costitutivi_txt")]
f2 = [f[:-4] for f in os.listdir("../ocr/scans")]

In [2]:
scans_filtered = [f for f in os.listdir("../ocr/scans_non_cost")]
#scans_filtered

Separa le scansioni di doc non costitutivi e le sposta in un'altra cartella

In [16]:
for i in scans_filtered:
    os.rename('../ocr/scans/'+i+'.pdf', '../ocr/scans_non_cost/'+i+'.pdf')
    print i + '.pdf'

4505787470001.pdf
4714406240001.pdf
4997803750001.pdf
4860752530001.pdf
4683987650001.pdf
4560302190001.pdf
4421722810001.pdf
4689046830001.pdf
4261082430001.pdf
4668945100001.pdf
5056150260001.pdf
4456562240001.pdf
4978896640001.pdf
4537974150001.pdf
4697990610001.pdf
4579574710001.pdf
4921510120001.pdf
4188044790001.pdf
4468807110001.pdf
4869227500001.pdf
4496214100001.pdf
4172764080001.pdf
4694363370001.pdf
4579834720001.pdf
4778908130001.pdf
4512947070001.pdf
4196866550001.pdf
4540944460001.pdf
4730473530001.pdf
4799104090001.pdf
4615984730001.pdf
4662468580001.pdf
5054653380001.pdf
4200887500001.pdf
4209333780001.pdf
4546585840001.pdf
4571293880001.pdf
4571119500001.pdf
4537745060001.pdf
4588024880001.pdf
4691794130001.pdf
4990833150001.pdf
4837917970001.pdf
4801275520001.pdf
4183076330001.pdf
4165619250001.pdf
4753170510001.pdf
5105989040001.pdf
4296542490001.pdf
4785770950001.pdf
4149729750001.pdf
4640708580001.pdf
4722475550001.pdf
4639972790001.pdf
4150931050001.pdf
4562063110

Converte i pdf in file di testo. sposta i fallimenti in altre cartelle.

In [8]:
def extract_text(f,out_dir, scans_dir, unhandled_dir, minlen):
    file = str(f.absolute())
    #print f.name
    try:
        text = textract.process(file).strip()
        if len(splitted_words(text)) <= minlen:
            print f.name, "is empty"
            copyfile(file, str(scans_dir + '/' + f.name))
        else:
            with open(out_dir + "/" + f.name[:-3] + 'txt', 'w') as outf:
                outf.write(text)
    except:
        print f.name
        copyfile(file, str(unhandled_dir + '/' + f.name))

def read_pdf_content(src_dir, out_dir, scans_dir, unhandled_dir, minlen):
    p = Path(src_dir)
    parallel_run.parallel_run_args(extract_text,((f, out_dir, scans_dir, unhandled_dir, minlen) for f in p.iterdir()))
    
    
def read_pdf_content_old(src_dir, out_dir, scans_dir, unhandled_dir, minlen):
    p = Path(src_dir)
    for f in p.iterdir():
        extract_text(f,out_dir, scans_dir, unhandled_dir, minlen)

In [51]:
read_pdf_content("../atti_costitutivi2", "../atti_costitutivi_txt", "../ocr/scans", "../unhadled_files", 100)

In [9]:
read_pdf_content('../atti_non_costitutivi2','../atti_non_costitutivi_txt2','../ocr/scans_non_cost','../ocr/unhandled_files', 100)

4505787470001.pdf is empty
4714406240001.pdf is empty
4997803750001.pdf is empty
4683987650001.pdf is empty
4668945100001.pdf is empty
4537974150001.pdf is empty
4697990610001.pdf is empty
4921510120001.pdf is empty
4496214100001.pdf is empty
4512947070001.pdf is empty
4540944460001.pdf is empty
4662468580001.pdf is empty
4546585840001.pdf is empty
4730473530001.pdf is empty
4537745060001.pdf is empty
4753170510001.pdf is empty
4640708580001.pdf is empty
4722475550001.pdf is empty
4639972790001.pdf is empty
4638008320001.pdf is empty
4660280570001.pdf is empty
4696710430001.pdf is empty
4523968290001.pdf is empty
4638406930001.pdf is empty
4597974300001.pdf is empty
4671664340001.pdf is empty
4908491280001.pdf is empty
4995874040001.pdf is empty
4604621780001.pdf is empty
4714635110001.pdf is empty
4513624600001.pdf is empty
4743627100001.pdf is empty
4546775440001.pdf is empty
4597360120001.pdf is empty
4521231100001.pdf is empty
4660234070001.pdf is empty
4734380260001.pdf is empty
4

# GOOGLE VISION API 

Extract jpg's from pdf's. Quick and dirty.

In [2]:
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

import codecs

In [3]:
def convert_to_pdf(f, res):
    pdf = file("../ocr/scans_non_cost/" + f, "rb").read()

    startmark = "\xff\xd8"
    startfix = 0
    endmark = "\xff\xd9"
    endfix = 2
    i = 0

    njpg = 0
    while True:
        istream = pdf.find("stream", i)
        if istream < 0:
            break
        istart = pdf.find(startmark, istream, istream+20)
        if istart < 0:
            i = istream+20
            continue
        iend = pdf.find("endstream", istart)
        if iend < 0:
            return
            raise Exception("Didn't find end of stream!")
        iend = pdf.find(endmark, iend-20)
        if iend < 0:
            return
            raise Exception("Didn't find end of JPG!")

        istart += startfix
        iend += endfix
        #print "JPG %d from %d to %d" % (njpg, istart, iend)
        res.append(pdf[istart:iend])
        #jpgfile = file("ocr/tempjpg/" + f[:-4] + "_%d.jpg" % njpg, "wb")
        #jpgfile.write(jpg)
        #jpgfile.close()
        #print ("ocr/tempjpg/" + f[:-4] + "_%d.jpg" % njpg)

        #njpg += 1
        i = iend

In [4]:
# Instantiates a client
client = vision.ImageAnnotatorClient()

def convert_to_txt(img):
    image = types.Image(content=img)
    # Performs label detection on the image file
    response = client.text_detection(image=image)
    labels = response.text_annotations
    
    try:
        return (labels[0].description)
    except:
        return ""   

In [ ]:
for i,f in enumerate(os.listdir("../ocr/scans_non_cost")):
    print i
    images = []
    convert_to_pdf(f, images)
    if len(images) > 0:
        res = ""
        for el in images:
            txt = convert_to_txt(el)
            res += txt
        if res != "":        
            with codecs.open("../ocr/scanned_non_costitutivi/" + f[:-4] + ".txt", "wb", 'utf-8') as outf:        
                outf.write(u'' + res)
        else:
            print f + " is empty"

In [ ]:
# Conversione elemento singolo (per effettuare test)

# Instantiates a client
client = vision.ImageAnnotatorClient()

def convert_to_txt(image):
    # Loads the image into memory
    with io.open(image, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    # Performs label detection on the image file
    response = client.text_detection(image=image)
    labels = response.text_annotations

    print('Labels:')
    for label in labels:
        print(label.description)